The goals of this notebook is to run optuna to find the best hyperparameters for object tracking in CellProfiler. 
I will do this by keeping segmentation the same and only changing the tracking parameters.
I will calculate a loss of the number of cells in a well over time compared to the ground truth of the number of cells in a well over time. 

In [2]:
import pathlib
import pprint
import sys

import optuna
import pandas as pd
import torch

sys.path.append("../CellProfiler_optuna_utils/")
from optuna_profiling_utils import adjust_cpipe_file, run_CytoTable

sys.path.append("../../utils/")
import cp_parallel
from cytotable import convert, presets
from pycytominer import annotate
from pycytominer.cyto_utils import output

sys.path.append("../../utils")
import sc_extraction_utils as sc_utils
from parsl.config import Config
from parsl.executors import HighThroughputExecutor

### CellProfiler paths and set up

In [ ]:
# set main output dir for all plates
output_dir = pathlib.Path("../analysis_output")
output_dir.mkdir(exist_ok=True, parents=True)

# directory where images are located within folders
images_dir = pathlib.Path("../../2.cellprofiler_ic_processing/illum_directory_test")

# path to plugins directory as one of the pipelines uses the RunCellpose plugin
plugins_dir = pathlib.Path(
    "/home/lippincm/Documents/CellProfiler/src/frontend/cellprofiler/modules/plugins"
)

# set the cpipe file
cpipe_template_file = pathlib.Path("../pipelines/analysis_4ch.cppipe").resolve(
    strict=True
)

In [ ]:
dict_of_inputs_for_cellprofiler = {
    "20231017ChromaLive_6hr_4ch_MaxIP": {
        "path_to_images": pathlib.Path(
            f"{images_dir}/20231017ChromaLive_6hr_4ch_MaxIP/"
        ).resolve(),
        "path_to_output": pathlib.Path(
            f"{output_dir}/20231017ChromaLive_6hr_4ch_MaxIP/"
        ).resolve(),
        "path_to_pipeline": pathlib.Path("../pipelines/analysis_4ch.cppipe").resolve(),
    },
}

# view the dictionary to assess that all info is added correctly
pprint.pprint(dict_of_inputs_for_cellprofiler, indent=4)

### CytoTable paths and set up

In [ ]:
# run CytoTable analysis for merged data
# type of file output from CytoTable (currently only parquet)
dest_datatype = "parquet"
# s1lite directory
source_dir = pathlib.Path("../analysis_output")
# directory where parquet files are saved to
output_dir = pathlib.Path("../data/converted_data")
output_dir.mkdir(exist_ok=True, parents=True)
dict_of_inputs_for_cytotable = {
    "run_20231004ChromaLive_6hr_4ch_MaxIP": {
        "source_path": pathlib.Path(
            f"{source_dir}/20231017ChromaLive_6hr_4ch_MaxIP/timelapse_4ch_analysis.sqlite"
        ).resolve(),
        "dest_path": pathlib.Path(
            f"{output_dir}/20231017ChromaLive_6hr_4ch_MaxIP.parquet"
        ).resolve(),
        "preset": """WITH Per_Image_Filtered AS (
                SELECT
                    Metadata_ImageNumber,
                    Image_Metadata_Well,
                    Image_Metadata_FOV,
                    Image_Metadata_Time,
                    Image_PathName_488_1
                    Image_PathName_488_2,
                    Image_PathName_561,
                    Image_PathName_DNA,
                    Image_FileName_488_1,
                    Image_FileName_488_2,
                    Image_FileName_561,
                    Image_FileName_DNA

                FROM
                    read_parquet('per_image.parquet')
                )
            SELECT
                *
            FROM
                Per_Image_Filtered AS per_image
            LEFT JOIN read_parquet('per_cytoplasm.parquet') AS per_cytoplasm ON
                per_cytoplasm.Metadata_ImageNumber = per_image.Metadata_ImageNumber
            LEFT JOIN read_parquet('per_cells.parquet') AS per_cells ON
                per_cells.Metadata_ImageNumber = per_cytoplasm.Metadata_ImageNumber
                AND per_cells.Metadata_Cells_Number_Object_Number = per_cytoplasm.Metadata_Cytoplasm_Parent_Cells
            LEFT JOIN read_parquet('per_nuclei.parquet') AS per_nuclei ON
                per_nuclei.Metadata_ImageNumber = per_cytoplasm.Metadata_ImageNumber
                AND per_nuclei.Metadata_Nuclei_Number_Object_Number = per_cytoplasm.Metadata_Cytoplasm_Parent_Nuclei
                """,
    },
}

### PyCytominer paths and set up

In [ ]:
# load in platemap file as a pandas dataframe
platemap_path = pathlib.Path("../../data/").resolve()

# directory where parquet files are located
data_dir = pathlib.Path("../data/converted_data")

# directory where the annotated parquet files are saved to
output_dir = pathlib.Path("../data/annotated_data")
output_dir.mkdir(exist_ok=True)

# dictionary with each run for the cell type
dict_of_inputs_for_pycytominer = {
    "run_20231017ChromaLive_6hr_4ch_MaxIP": {
        "source_path": pathlib.Path(
            f"{data_dir}/20231017ChromaLive_6hr_4ch_MaxIP.parquet"
        ).resolve(strict=True),
        "platemap_path": pathlib.Path(f"{platemap_path}/platemap_6hr_4ch.csv").resolve(
            strict=True
        ),
    }
}

### Optuna set up and search space

In [ ]:
# create an otuna parameter search space
# define the search space in a dictionary format
# defined as [min, max] for each parameter
parameters_search_space = {
    "std_search_radius": [1, 100],
    "search_radius_min": [1, 10],
    "search_radius_max": [15, 100],
    "gap_closing_cost": [10, 100],
    "split_alternative_cost": [10, 100],
    "merge_alternative_cost": [10, 100],
    "Maximum_gap_displacement": [1, 15],
    "Maximum_split_score": [10, 100],
    "Maximum_merge_score": [10, 100],
    "Maximum_temporal_gap": [1, 5],
    "Mitosis_alternative_cost": [10, 150],
    "Maximum_mitosis_distance": [10, 150],
}

In [ ]:
# parameters for the optimization


def objective(
    trial: optuna.Trial,
    search_space_parameters: dict,
    dict_of_inputs_for_cellprofiler: dict,
    dict_of_inputs_for_cytotable: dict,
    cpipe_template_file: pathlib.Path,
    plugins_dir: pathlib.Path,
):
    dictionary_of_selected_parameters = {
        "std_search_radius": trial.suggest_int(
            "std_search_radius",
            search_space_parameters["std_search_radius"][0],
            search_space_parameters["std_search_radius"][1],
        ),
        "search_radius_min": trial.suggest_int(
            "search_radius_min",
            search_space_parameters["search_radius_min"][0],
            search_space_parameters["search_radius_min"][1],
        ),
        "search_radius_max": trial.suggest_int(
            "search_radius_max",
            search_space_parameters["search_radius_max"][0],
            search_space_parameters["search_radius_max"][1],
        ),
        "gap_closing_cost": trial.suggest_int(
            "gap_closing_cost",
            search_space_parameters["gap_closing_cost"][0],
            search_space_parameters["gap_closing_cost"][1],
        ),
        "split_alternative_cost": trial.suggest_int(
            "split_alternative_cost",
            search_space_parameters["split_alternative_cost"][0],
            search_space_parameters["split_alternative_cost"][1],
        ),
        "merge_alternative_cost": trial.suggest_int(
            "merge_alternative_cost",
            search_space_parameters["merge_alternative_cost"][0],
            search_space_parameters["merge_alternative_cost"][1],
        ),
        "Maximum_gap_displacement": trial.suggest_int(
            "Maximum_gap_displacement",
            search_space_parameters["Maximum_gap_displacement"][0],
            search_space_parameters["Maximum_gap_displacement"][1],
        ),
        "Maximum_split_score": trial.suggest_int(
            "Maximum_split_score",
            search_space_parameters["Maximum_split_score"][0],
            search_space_parameters["Maximum_split_score"][1],
        ),
        "Maximum_merge_score": trial.suggest_int(
            "Maximum_merge_score",
            search_space_parameters["Maximum_merge_score"][0],
            search_space_parameters["Maximum_merge_score"][1],
        ),
        "Maximum_temporal_gap": trial.suggest_int(
            "Maximum_temporal_gap",
            search_space_parameters["Maximum_temporal_gap"][0],
            search_space_parameters["Maximum_temporal_gap"][1],
        ),
        "Mitosis_alternative_cost": trial.suggest_int(
            "Mitosis_alternative_cost",
            search_space_parameters["Mitosis_alternative_cost"][0],
            search_space_parameters["Mitosis_alternative_cost"][1],
        ),
        "Maximum_mitosis_distance": trial.suggest_int(
            "Maximum_mitosis_distance",
            search_space_parameters["Maximum_mitosis_distance"][0],
            search_space_parameters["Maximum_mitosis_distance"][1],
        ),
    }

    for key, value in dictionary_of_selected_parameters.items():
        print(f"{key}: {value}")
    print(trial.number)

    # run the pipeline adjustment function
    adjusted_cpipe_file = adjust_cpipe_file(
        trial_number=trial.number,
        cpipe_file_path=cpipe_template_file,
        parameters_dict=dictionary_of_selected_parameters,
    )

    # update the cellprofiler dictionary with the new pipeline
    dict_of_inputs_for_cellprofiler["20231017ChromaLive_6hr_4ch_MaxIP"][
        "path_to_pipeline"
    ] = adjusted_cpipe_file

    run_name = f"run_name_{trial.number}"

    # run cellprofiler pipeline
    cp_parallel.run_cellprofiler_parallel(
        plate_info_dictionary=dict_of_inputs_for_cellprofiler,
        run_name=run_name,
        plugins_dir=plugins_dir,
    )

    # run CytoTable analysis for merged data
    run_CytoTable(cytotable_dict=dict_of_inputs_for_cytotable)

,Metadata_Nuclei_TrackObjects_Label_50,Metadata_number_of_singlecells,Metadata_Well,Metadata_FOV,Metadata_Time
0,1,6743,E-10,0001,0001
1,3,6743,E-10,0001,0001
2,4,6743,E-10,0001,0001
3,5,6743,E-10,0001,0001
4,6,6743,E-10,0001,0001


,Metadata_Well,Metadata_Nuclei_TrackObjects_Label_50
0,C-02,7160


In [65]:
def retrieve_cell_count():

    path_df = pathlib.Path(
        "../../4.process_CP_features/data/annotated_data/run_20231017ChromaLive_6hr_4ch_MaxIP_sc.parquet"
    ).resolve(strict=True)

    # read in the dataframe
    df = pd.read_parquet(
        path_df,
        columns=[
            "Metadata_Nuclei_TrackObjects_Label_50",
            "Metadata_number_of_singlecells",
            "Metadata_Well",
            "Metadata_FOV",
            "Metadata_Time",
        ],
    )

    df.head()

    # sum up the number of unique single cells per well per time point per FOV using the Nuclei objects label
    df_actual_cell_counts = (
        df.groupby(["Metadata_Well", "Metadata_FOV", "Metadata_Time"])
        .Metadata_Nuclei_TrackObjects_Label_50.nunique()
        .reset_index()
    )
    df_actual_cell_counts = (
        df_actual_cell_counts.groupby(
            [
                "Metadata_Well",
                # "Metadata_Time"
            ]
        )
        .Metadata_Nuclei_TrackObjects_Label_50.sum()
        .reset_index()
    )

    # drop all columns except for Metadata_Well and number of single cells
    target_cell_count = df.copy()
    target_cell_count = target_cell_count[
        ["Metadata_Well", "Metadata_number_of_singlecells"]
    ]
    target_cell_count.drop_duplicates(inplace=True)

    # sort byt well
    target_cell_count.sort_values(by="Metadata_Well", inplace=True)
    target_cell_count.reset_index(drop=True, inplace=True)

    # get the Metadata_Nuclei_TrackObjects_Label_50 column from the actual cell counts dataframe
    actual_cell_counts = df_actual_cell_counts[
        "Metadata_Nuclei_TrackObjects_Label_50"
    ].to_list()
    target_cell_counts = target_cell_count["Metadata_number_of_singlecells"].to_list()

    return actual_cell_counts, target_cell_counts

,Metadata_Well,Metadata_number_of_singlecells
0,C-02,7914
